# Redis Product Catalog Tutorial

## Product Catalog Use Case

In the last session several common Redis use cases were discussed. Now, lets consider using Redis as our primary database for a product catalog for a new online store. It will need to store product details including a name, description, vendor, price, main category and some images.


## Requirements:

* Product information stored in the database should include: name, description, vendor, price, category, images associated with that product.
* Ability to create/update/delete product details
* Ability to find product by ID
* Ability to find products in category X
* Ability to find product by its name or part of its name


## Logical Data Model:

The logical data model is separate from the DBMS being used.
It defines the structure of data elements and to set relationships between them.

* Product Image
    * Id: Number
    * Value: Binary

* Product
    * Id: Number
    * Name: String
    * Description: String
    * Vendor: String
    * Price: Number
    * Currency: String
    * MainCategory: Category (1)
    * Images: Image (0..n)
 
* Category
    * id: Number
    * Name: String
    * Products: Product (0..n)

## Redis Data Model:

Different than the logical model, see below

In [1]:
# first import and get to the path of the repo
import os
from pathlib import Path
current_dir = Path.cwd()
path = current_dir.parent
print(path)
os.chdir(path)
current_dir = Path.cwd()
print(current_dir)

/Users/brandonamos/Documents/repos/redisprodcatalog
/Users/brandonamos/Documents/repos/redisprodcatalog


In [2]:
# import from productcatalog
from productcatalog.product_catalog_create import *
from productcatalog.product_catalog_update import *
from productcatalog.product_catalog_search import *

In [12]:
# example product dict list
product_dict_list = [
    {
    "name": 'Doritos Spicy Nacho',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'HEB',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Doritos Salsa Verde',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Costco',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Doritos Flaming Hot Nacho Cheese',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Kroger',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Doritos Cool Ranch',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Doritos',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Tostitos Hint of Lime',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Costco',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Tostitos Scoops',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Kroger',
    "price": 4.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Tostitos Original',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'Kroger',
    "price": 4.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Cheetos XXtra Flaming Hot',
    "description": 'Cheese puff snack',
    "vendor": 'HEB',
    "price": 2.99,
    "currency": 'dollars',
    "category": 'Cheese_Puff',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Cheetos Baked Crunchy',
    "description": 'Cheese puff snack',
    "vendor": 'HEB',
    "price": 2.99,
    "currency": 'dollars',
    "category": 'Cheese_Puff',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Cheetos Original',
    "description": 'Cheese puff snack',
    "vendor": 'Kroger',
    "price": 2.99,
    "currency": 'dollars',
    "category": 'Cheese_Puff',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Kale',
    "description": 'Leafy Greens',
    "vendor": 'HEB',
    "price": 5.50,
    "currency": 'dollars',
    "category": 'Vegetable',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Iceburg Lettuce',
    "description": 'Leafy Greens',
    "vendor": 'Kroger',
    "price": 1.50,
    "currency": 'dollars',
    "category": 'Vegetable',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Butter Lettuce',
    "description": 'Leafy Greens',
    "vendor": 'HEB',
    "price": 3.50,
    "currency": 'dollars',
    "category": 'Vegetable',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Pork',
    "description": 'The Meats',
    "vendor": 'HEB',
    "price": 3.50,
    "currency": 'dollars',
    "category": 'Meat',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Chicken',
    "description": 'The Meats',
    "vendor": 'HEB',
    "price": 5.50,
    "currency": 'dollars',
    "category": 'Meat',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Beef',
    "description": 'The Meats',
    "vendor": 'HEB',
    "price": 9.50,
    "currency": 'dollars',
    "category": 'Meat',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Tuna',
    "description": 'Fishy',
    "vendor": 'HEB',
    "price": 5.50,
    "currency": 'dollars',
    "category": 'Fish',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Whale',
    "description": 'Fishy',
    "vendor": 'HEB',
    "price": 20.50,
    "currency": 'dollars',
    "category": 'Fish',
    "images": [1,2,3,4,5,6,7,8]
    },
    {
    "name": 'Snapper',
    "description": 'Fishy',
    "vendor": 'HEB',
    "price": 12,
    "currency": 'dollars',
    "category": 'Fish',
    "images": [1,2,3,4,5,6,7,8]
    }
]

## Flush Db to Start

In [4]:
# lets start out by flushing the db
import redis

# connect redis to localhost and port
redis = redis.Redis(host='localhost', port=6379, db=0)
redis.flushdb()

True

## Create a Product

Lets create a product and put it in our redis db.

Below is a example of what a product should look like in the database.
While Redis is schemaless, I have required a schema in this db.

Each product should have the following:
* Name: (str)
    * **must be unique, no duplicate names for now**
* Description: (str)
* Vendor: (str)
* Price: (a int or float)
* Currency: (str)
* Category: (str)
* Images: (a list of binary images)
    * I have just used numbers for now, but it could accept binary objects.

In [5]:
# example dict
product_dict = {
    "name": 'Doritos Nacho Cheese',
    "description": 'Flavored Tortilla Chip',
    "vendor": 'HEB',
    "price": 3.99,
    "currency": 'dollars',
    "category": 'Chip',
    "images": [1,2,3,4,5,6,7,8]
    }

# Now put the product dict into ProductCatalogCreate 
# to generate the Redis Datastructures in the Redis DB
product_obj = ProductCatalogCreate(product_dict) # get product object
print('This takes the dict and puts it into a dataclass object')
print('')
print('product.data:')
print(product_obj.data)
print('')
print('product dataclass obj:')
print(product_obj.product_obj)

# create the new product and put it in the db
product_obj.generate_product_catalog()

This takes the dict and puts it into a dataclass object

product.data:
{'name': 'Doritos Nacho Cheese', 'description': 'Flavored Tortilla Chip', 'vendor': 'HEB', 'price': 3.99, 'currency': 'dollars', 'category': 'Chip', 'images': [1, 2, 3, 4, 5, 6, 7, 8]}

product dataclass obj:
ProductCreate(name='Doritos Nacho Cheese', description='Flavored Tortilla Chip', vendor='HEB', price=3.99, currency='dollars', category='Chip', productId=None, images=array([1, 2, 3, 4, 5, 6, 7, 8]), redisSession=[Redis<ConnectionPool<Connection<host=localhost,port=6379,db=0>>>])


**Product Obj**

The product obj contains all the data from the dict and also a redis client session.

### What else does it create?

In [6]:
redis = product_obj.product_obj.redisSession[0]

# do a redis scan to check out the db contents
redis.scan()

(0,
 [b'category:Chip',
  b'product:1:images',
  b'product-names',
  b'product_id_incr',
  b'product:1',
  b'product-name-index'])

### Product Hash
We get a Redis Hash of a product id. **product:1**

The hash contains all the product information.

The product Id is generated using a Redis Incr command, that will 
continue to increment up by 1 for each new product in the db.

In [7]:
redis.hgetall('product:1')

{b'name': b'Doritos Nacho Cheese',
 b'description': b'Flavored Tortilla Chip',
 b'vendor': b'HEB',
 b'price': b'3.99',
 b'currency': b'dollars',
 b'category': b'Chip'}

### Product Images

Since Redis does not allow for nested data structures a product image index
was generated, this is the product id and images text concat together. (product:1:images)

The product images are stored in a list. The code allows for 4 product images and drops any extra images.

In [8]:
redis.lrange('product:1:images',0,-1)

[b'1', b'2', b'3', b'4']

### Category Index

We create a set for the category. It is given an id of 'category:(the category from the dict)'

The set will hold all the products in that category, with no duplicates

In [9]:
redis.smembers('category:Chip')

{b'product:1'}

### Product Names:

A set of all unique product names is generated just to ensure no duplicate names are used
when attempting to put a new product in the db.

In [10]:
redis.smembers('product-names')

{b'Doritos Nacho Cheese'}

### Product Names Index:

A secondary index of product names and the product Ids is generated so that it can be
used in a Name search when you do not know the Id.

In [11]:
redis.hgetall('product-name-index')

{b'Doritos Nacho Cheese': b'product:1'}

## Now that we have seen all the data structures, lets put some data in the DB

In [13]:
# Loop through product dict list and put in the db.
for item in product_dict_list:
    
    product_obj = ProductCatalogCreate(item) # get product object
    product_obj.generate_product_catalog() # create products

In [14]:
# do a quick scan to check if new products are present.
redis.scan()

(12,
 [b'product:2',
  b'product:7',
  b'product:9',
  b'product:18:images',
  b'product:3:images',
  b'category:Chip',
  b'product:18',
  b'product:11:images',
  b'product:1:images',
  b'product:9:images',
  b'product:11'])

# Now lets find products, update products, and delete them:

### Find product by product id:

In [15]:
product_id = 'product:5'

In [16]:
# product find
product_dict = {
    "productId": product_id
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_by_product_id()

{b'name': b'Doritos Cool Ranch',
 b'description': b'Flavored Tortilla Chip',
 b'vendor': b'Doritos',
 b'price': b'3.99',
 b'currency': b'dollars',
 b'category': b'Chip'}

### Lets update this product and its contents:

First lets do a full update, change everything.

In [17]:
# example dict
product_dict = {
    "productId": product_id,
    "name": 'Updated Product Name',
    "description": 'Updated Desc',
    "vendor": 'Updated Vendor',
    "price": 100,
    "currency": 'Euros',
    "category": 'NewCat',
    "images": [233,2450]
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.update_product() # update the product info

In [18]:
# Lets find the updated product
# product find
product_dict = {
    "productId": product_id
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_by_product_id()

{b'name': b'Updated Product Name',
 b'description': b'Updated Desc',
 b'vendor': b'Updated Vendor',
 b'price': b'100.0',
 b'currency': b'Euros',
 b'category': b'NewCat'}

### Nice, but what about the data in those other data structures? Doesnt it need to change?

Dont worry, that is all handled for you.
Lets check it out

In [19]:
# check out the product:
print('check out the product:')
print(redis.hgetall(product_id))

print("")
print('check out the new product images: (see how it has pushed older images out)')
print(redis.lrange(f"{product_id}:images",0,-1))

print('')
print('Check out the original category: (notice the product id is gone)')
print(redis.smembers('category:Chip'))

#print('')
#print(redis.smembers('product-names'))

print('check out the new catagory, notice a new product!')
print(redis.smembers('category:NewCat'))

check out the product:
{b'name': b'Updated Product Name', b'description': b'Updated Desc', b'vendor': b'Updated Vendor', b'price': b'100.0', b'currency': b'Euros', b'category': b'NewCat'}

check out the new product images: (see how it has pushed older images out)
[b'233', b'2450', b'1', b'2']

Check out the original category: (notice the product id is gone)
{b'product:2', b'product:4', b'product:6', b'product:1', b'product:8', b'product:3', b'product:7'}
check out the new catagory, notice a new product!
{b'product:5'}


### Lets update 1 or 2 fields:

Sometimes you might just want to update a couple of fields. Lets do that

In [20]:
# example dict
product_dict = {
    "productId": product_id,
    "name": 'Updated Again',
    "category": 'Chip',
    "images": ['new']
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.update_product() # update the product info

# Lets find the updated product
# product find
product_dict = {
    "productId": product_id
}
product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_by_product_id()

{b'name': b'Updated Again',
 b'description': b'Updated Desc',
 b'vendor': b'Updated Vendor',
 b'price': b'100.0',
 b'currency': b'Euros',
 b'category': b'Chip'}

In [21]:
# check out the product:
print('check out the product:')
print(redis.hgetall(product_id))

print("")
print('check out the new product images: (see how it has pushed older images out)')
print(redis.lrange(f"{product_id}:images",0,-1))

print('')
print('Check out the original category: (notice the product id is back!)')
print(redis.smembers('category:Chip'))

#print('')
#print(redis.smembers('product-names'))

print('check out the previous New cat, notice it is empty')
print(redis.smembers('category:NewCat'))

check out the product:
{b'name': b'Updated Again', b'description': b'Updated Desc', b'vendor': b'Updated Vendor', b'price': b'100.0', b'currency': b'Euros', b'category': b'Chip'}

check out the new product images: (see how it has pushed older images out)
[b'new', b'233', b'2450', b'1']

Check out the original category: (notice the product id is back!)
{b'product:2', b'product:4', b'product:6', b'product:1', b'product:8', b'product:3', b'product:7', b'product:5'}
check out the previous New cat, notice it is empty
set()


### We can also find images by product id only:

In [22]:
# product images find
product_dict = {
    "productId": 'product:1'
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_images_by_product_id()

[b'1', b'2', b'3', b'4']

### Find products in a Category:

In [23]:
# category find
product_dict = {
    "category": 'Chip'
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_products_in_category()

category:Chip


{b'product:1',
 b'product:2',
 b'product:3',
 b'product:4',
 b'product:5',
 b'product:6',
 b'product:7',
 b'product:8'}

In [24]:
# category find
product_dict = {
    "category": 'Cheese_Puff'
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_products_in_category()

category:Cheese_Puff


{b'product:10', b'product:11', b'product:9'}

### What happens if I dont know the Product Id? 

Dont worry, you can find products using the secondary index in Redis
Just by the product Name

In [25]:
# find product by its name or part of its name
product_name = 'Doritos Flaming Hot Nacho Cheese'
#product_name = 'Doritos Nacho Cheese'
product_dict = {
    "name": product_name
}

product_obj = ProductCatalogSearch(product_dict) # get product object
product_obj.find_products_by_name()

{b'name': b'Doritos Flaming Hot Nacho Cheese',
 b'description': b'Flavored Tortilla Chip',
 b'vendor': b'Kroger',
 b'price': b'3.99',
 b'currency': b'dollars',
 b'category': b'Chip'}

### What about if I only know the partial name?

Use Full Text Search!

# Set Up RediSearch

This is not my python app yet, for now, just set it up manually

We will want to create a RediSearch Index, lets make it look like this one

In [26]:
# RediSearch Index
"""
FT.CREATE 
idx:product ON hash 
    PREFIX 1 "product:" 
    SCHEMA 
    name TEXT SORTABLE
    description TEXT SORTABLE
    vendor TAG SORTABLE
    price NUMERIC SORTABLE
    main-category TAG SORTABLE    
"""
print('')

There are a couple of ways to do this.

You can execute the following query in the redis-cli:

In [27]:
# FT.CREATE idx:product ON hash PREFIX 1 "product:" SCHEMA name TEXT SORTABLE description TEXT SORTABLE vendor TAG SORTABLE price NUMERIC SORTABLE category TAG SORTABLE

Or you can set it up with redisearch-py package

In [28]:
from redisearch import Client, TextField, NumericField, TagField, IndexDefinition, Query

# Creating a client with a given index name
client = Client("idx:product")

# IndexDefinition is available for RediSearch 2.0+
definition = IndexDefinition(prefix=['product:'])

# Creating the index definition and schema
client.create_index((TextField("name", sortable=True), 
                     TextField("description", sortable=True), 
                     TagField("vendor"), 
                     NumericField("price", sortable=True), 
                     TagField("category")), definition=definition)

'OK'

### Lets Run Some queries:

Here is a list of queries you could run from the cli

In [29]:
# FT.SEARCH idx:product "Doritos" 

# FT.SEARCH idx:product "Flaming" RETURN 2 price category 

#FT.SEARCH idx:product "@vendor:{Costco|Kroger}" RETURN 2 name price
    
#FT.SEARCH idx:product "@category:{Meat|Fish} @price:[9 15]" RETURN 3 name price category  

In [30]:
# Simple search using the rediSearch python package
res = client.search("Doritos")

print('total Items:')
print(res.total)
print('')
for item in res.docs:
    print(item.name)
    print(item.vendor)

total Items:
4

Doritos Nacho Cheese
HEB
Doritos Salsa Verde
Costco
Doritos Flaming Hot Nacho Cheese
Kroger
Doritos Spicy Nacho
HEB


### Use the Cursor:
Or we can just put them directy into the redis cursor:

In [31]:
# FT.SEARCH idx:product "Doritos" 
redis.execute_command('ft.search', 'idx:product', 'Flaming')

[2,
 b'product:4',
 [b'name',
  b'Doritos Flaming Hot Nacho Cheese',
  b'description',
  b'Flavored Tortilla Chip',
  b'vendor',
  b'Kroger',
  b'price',
  b'3.99',
  b'currency',
  b'dollars',
  b'category',
  b'Chip'],
 b'product:9',
 [b'name',
  b'Cheetos XXtra Flaming Hot',
  b'description',
  b'Cheese puff snack',
  b'vendor',
  b'HEB',
  b'price',
  b'2.99',
  b'currency',
  b'dollars',
  b'category',
  b'Cheese_Puff']]

In [33]:
# FT.SEARCH idx:product "Flaming" RETURN 2 price category
redis.execute_command('ft.search', 'idx:product', 'Flaming')

[2,
 b'product:4',
 [b'name',
  b'Doritos Flaming Hot Nacho Cheese',
  b'description',
  b'Flavored Tortilla Chip',
  b'vendor',
  b'Kroger',
  b'price',
  b'3.99',
  b'currency',
  b'dollars',
  b'category',
  b'Chip'],
 b'product:9',
 [b'name',
  b'Cheetos XXtra Flaming Hot',
  b'description',
  b'Cheese puff snack',
  b'vendor',
  b'HEB',
  b'price',
  b'2.99',
  b'currency',
  b'dollars',
  b'category',
  b'Cheese_Puff']]

In [34]:
#FT.SEARCH idx:product "@vendor:{Costco|Kroger}" RETURN 2 name price
redis.execute_command('ft.search', 'idx:product', "@vendor:{Costco|Kroger}", 
                      "RETURN", "2", "name", "price")

[7,
 b'product:3',
 [b'name', b'Doritos Salsa Verde', b'price', b'3.99'],
 b'product:6',
 [b'name', b'Tostitos Hint of Lime', b'price', b'3.99'],
 b'product:8',
 [b'name', b'Tostitos Original', b'price', b'4.99'],
 b'product:13',
 [b'name', b'Iceburg Lettuce', b'price', b'1.5'],
 b'product:4',
 [b'name', b'Doritos Flaming Hot Nacho Cheese', b'price', b'3.99'],
 b'product:11',
 [b'name', b'Cheetos Original', b'price', b'2.99'],
 b'product:7',
 [b'name', b'Tostitos Scoops', b'price', b'4.99']]

In [35]:
#FT.SEARCH idx:product "@category:{Meat|Fish} @price:[9 15]" RETURN 3 name price category
redis.execute_command('ft.search', 'idx:product', "@category:{Meat|Fish} @price:[9 15]",
                      "RETURN", "3", "name", "price", "category")

[2,
 b'product:20',
 [b'name', b'Snapper', b'price', b'12', b'category', b'Fish'],
 b'product:17',
 [b'name', b'Beef', b'price', b'9.5', b'category', b'Meat']]

# Delete A Product
### Now I want to delete a product.

This will delete the product and remove it from all the other data structures it was related to

In [36]:
# Delete Product by Product ID
product_dict = {
    "productId": product_id
}

product_obj = ProductCatalogUpdate(product_dict) # get product object
product_obj.delete_product()

In [37]:
# check out the product:
print('check out the product: (It is gone!)')
print(redis.hgetall(product_id))

print("")
print('check out the new product images: (They are gone!)')
print(redis.lrange(f"{product_id}:images",0,-1))

print('')
print('Check out the original category: (The product id is gone!)')
print(redis.smembers('category:Chip'))

print('')
print('The product name is gone! you can use it again')
print(redis.smembers('product-names'))

check out the product: (It is gone!)
{}

check out the new product images: (They are gone!)
[]

Check out the original category: (The product id is gone!)
{b'product:2', b'product:4', b'product:6', b'product:1', b'product:8', b'product:3', b'product:7'}

The product name is gone! you can use it again
{b'Cheetos Original', b'Doritos Flaming Hot Nacho Cheese', b'Tostitos Original', b'Chicken', b'Doritos Spicy Nacho', b'Tostitos Scoops', b'Tostitos Hint of Lime', b'Iceburg Lettuce', b'Snapper', b'Cheetos Baked Crunchy', b'Tuna', b'Kale', b'Butter Lettuce', b'Doritos Salsa Verde', b'Doritos Nacho Cheese', b'Pork', b'Whale', b'Cheetos XXtra Flaming Hot', b'Beef'}
